In [16]:
self.complete_dataset_xml_dir = os.path.join(self.dcc_directory,"complete_dataset",'chips_positive_xml') 
os.makedirs(self.complete_dataset_xml_dir, exist_ok=True) #directory to hold entire dataset annotations
self.complete_dataset_chips_dir = os.path.join(self.dcc_directory,"complete_dataset","chips_positive") 
os.makedirs(self.complete_dataset_chips_dir, exist_ok=True) #directory to hold xml files

NameError: name 'self' is not defined

In [20]:
parent_directory = "Z:\\verification_set2\\unverified_images\\student_reviewed_unverified_images"
ap.list_of_sub_directories(parent_directory)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Z:\\verification_set2\\unverified_images\\student_reviewed_unverified_images'

In [ ]:
sub_directories = []
for annotator_directory in ap.list_of_sub_directories(args.parent_directory):
    for root,dirs,files in os.walk(annotator_directory):
        if "chips" in dirs: #excludes the annotations from Jackson that need to be reformated
            sub_directories.append(root)

### Move the annotations + images 
counter_annotations = 0
counter_images = 0

In [14]:
set_number = '2'
home_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk"
folder_names = ["Niculescu_Sunny_Poonacha"]                            
annotator_allocation = ['Mia','Tang']
tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'

In [48]:
positive_xml = os.listdir("//oit-nas-fe13dc.oit.duke.edu/data_commons-borsuk/complete_dataset/chips_positive_xml")
for i in range(len(positive_xml)):
    positive_xml[i] = positive_xml[i].split(".")[0]
positive = os.listdir("//oit-nas-fe13dc.oit.duke.edu/data_commons-borsuk/complete_dataset/chips_positive")
for i in range(len(positive)):
    positive[i] = positive[i].split(".")[0]

In [50]:
main_list = list(set(positive_xml) - set(positive))
main_list

['ca_060cm_2018_34117_m_3411760_sw_11_060_20180822_000001']

In [59]:
unique, frequency = np.unique(np.load(tracker_file_path)[:,1], 
                               return_counts = True) 
print(len(unique))
for i in range(len(frequency)):
    if frequency[i]>1:
        print(unique[i])

19698
al_60cm_2019_34087_m_3408711_sw_16_060_20190908_000265.jpg
al_60cm_2019_34087_m_3408711_sw_16_060_20190908_000266.jpg
al_60cm_2019_34087_m_3408711_sw_16_060_20190908_000290.jpg
al_60cm_2019_34087_m_3408711_sw_16_060_20190908_000291.jpg
al_60cm_2019_34087_m_3408711_sw_16_060_20190908_000395.jpg
al_60cm_2019_34087_m_3408711_sw_16_060_20190908_000486.jpg
al_60cm_2019_34087_m_3408711_sw_16_060_20190908_000490.jpg
al_60cm_2019_34087_m_3408711_sw_16_060_20190908_000511.jpg
al_60cm_2019_34087_m_3408719_nw_16_060_20190908_000010.jpg
al_60cm_2019_34087_m_3408719_nw_16_060_20190908_000011.jpg
al_60cm_2019_34087_m_3408719_nw_16_060_20190908_000142.jpg
al_60cm_2019_34087_m_3408719_nw_16_060_20190908_000162.jpg
al_60cm_2019_34087_m_3408719_nw_16_060_20190908_000244.jpg
al_60cm_2019_34088_m_3408807_nw_16_060_20191012_000205.jpg
al_60cm_2019_34088_m_3408807_nw_16_060_20191012_000229.jpg
al_60cm_2019_34088_m_3408807_nw_16_060_20191012_000230.jpg
al_60cm_2019_34088_m_3408807_nw_16_060_20191012_00

In [57]:
positive = os.listdir("//oit-nas-fe13dc.oit.duke.edu/data_commons-borsuk/complete_dataset/chips_positive")
positive_recorded = list(np.unique(tile_img_annotation_annotator[:,1]))
main_list = list(set(positive_recorded) - set(positive))
main_list

['wy_60cm_2019_41110_m_4111025_ne_12_060_20190728_000416.jpg',
 'oh_60cm_2019_41083_m_4108321_sw_17_060_20190825_000041.jpg',
 'la_60cm_2019_31091_m_3109125_nw_15_060_20190902_000418.jpg',
 'il_60cm_2019_41087_m_4108711_nw_16_060_20190802_000479.jpg',
 'wy_60cm_2019_41107_m_4110716_sw_13_060_20190723_000304.jpg',
 'oh_60cm_2019_41081_m_4108144_se_17_060_20191012_000451.jpg',
 'oh_60cm_2019_41083_m_4108321_sw_17_060_20190825_000042.jpg',
 'oh_60cm_2019_41083_m_4108353_sw_17_060_20191008_000440.jpg',
 'la_60cm_2019_30093_m_3009354_sw_15_060_20190905_000014.jpg',
 'la_60cm_2019_31091_m_3109129_sw_15_060_20190725_000246.jpg',
 'il_60cm_2019_41087_m_4108710_sw_16_060_20190809_000360.jpg',
 'la_60cm_2019_31091_m_3109129_se_15_060_20190725_000330.jpg',
 'oh_60cm_2019_41081_m_4108161_nw_17_060_20191012_000011.jpg',
 'oh_60cm_2019_41081_m_4108145_sw_17_060_20191012_000228.jpg',
 'il_60cm_2019_41087_m_4108701_ne_16_060_20190809_000465.jpg',
 'oh_60cm_2019_41083_m_4108321_sw_17_060_20190825_00012

In [ ]:
#set3
#folder_names_set3 = ["anyone",  ]       # Tang#2         Poonacha#2   Sunny#2                         
#annotator_allocation_set3 = [['Josh','Jaewon',"Shen",'Katpally']]